In [2]:
import pandas as pd
# from IPython.display import display
pd.set_option("display.max_colwidth", None)

## Helper Functions

In [3]:
def save_as_csv(dataframe, file_name):
    import datetime
    date_info = datetime.datetime.now().strftime("%d-%m-%y_%H:%M:%S")
    dataframe.to_csv(f"scraping_analysis/{date_info}_{file_name}.csv", index=False)

## Code

In [4]:
df = pd.read_json("scrape_archive/full_scrape.json")

In [123]:
# wie viele in jeder spalte ausser "url", null oder "" enthalten, und im string der spalte "url" das wort "kunst" vorkommt
df.loc[
    (
        (df.loc[:, ~df.columns.isin(["url"])] == "") |
        (df.loc[:, ~df.columns.isin(["url"])].isnull())
    ).all(axis=1)
    & df["url"].str.contains("https")
].shape[0]

2306

In [50]:
def count_string_occurences_in_url(pandas_df):
    things_to_check = [
        "https",
        "kunst",
        "generalagenturen",
        "jobs",
        "report",
        "vcard",
        "mobiliare",
        "mobiliere",
        "hub",
        "medienmitteilungen",
        "gallerySlide",
    ]  # https to get 100%
    results = []
    for item in things_to_check:
        item_count = pandas_df[pandas_df["url"].str.contains(item)].shape[0]
        pct = round(item_count / pandas_df.shape[0] * 100, 2)
        count_empty = pandas_df.loc[
            ((pandas_df.loc[:, ~pandas_df.columns.isin(["url"])] == "")
                | (pandas_df.loc[:, ~pandas_df.columns.isin(["url"])].isnull())
            ).all(axis=1)
            & pandas_df["url"].str.contains(item)
        ].shape[0]
        count_empty_except_title = pandas_df.loc[
            ((pandas_df.loc[:, ~pandas_df.columns.isin(["url", "page_title"])] == "")
                | (pandas_df.loc[:, ~pandas_df.columns.isin(["url", "page_title"])].isnull())
            ).all(axis=1)
            & pandas_df["url"].str.contains(item)
        ].shape[0]
        results.append(
            {
                "item_name": item,
                "item_count": item_count,
                "%": pct,
                "empty_except_url": count_empty,
                "all_empty_%": round(count_empty / item_count * 100, 2),
                "empty_cols_except_title_&_url": count_empty_except_title,
                "title_&_url_empty_%": round(count_empty_except_title / item_count * 100, 2),

            }
        )
    df = pd.DataFrame(results).sort_values('item_count', ascending=[False])
    return df

In [51]:
count_string_occurences_in_url(df)
# save_as_csv(count_string_occurences_in_url(df), "evaluation_occurrences_of_words_in_url_after_first_scrape")

,item_name,item_count,%,empty_except_url,all_empty_%,empty_cols_except_title_&_url,title_&_url_empty_%
0,https,4809,100.00,2306,47.95,3305,68.73
1,kunst,2046,42.55,1851,90.47,1969,96.24
9,medienmitteilungen,765,15.91,0,0.00,292,38.17
10,gallerySlide,656,13.64,0,0.00,655,99.85
5,vcard,154,3.20,154,100.00,154,100.00
2,generalagenturen,151,3.14,5,3.31,139,92.05
3,jobs,134,2.79,128,95.52,128,95.52
4,report,101,2.10,101,100.00,101,100.00
7,mobiliere,86,1.79,1,1.16,54,62.79
8,hub,26,0.54,20,76.92,22,84.62


In [185]:
not_empty = df.loc[
    (
        (df.loc[:, ~df.columns.isin(["url"])] != "") |
        (df.loc[:, ~df.columns.isin(["url"])].notnull())
    ).all(axis=1)
    & ~((df["url"].str.contains("kunst")) | (df["url"].str.contains("vcard"))| (df["url"].str.contains("jobs"))| (df["url"].str.contains("report"))) 
]
not_empty.shape[0]
# save_as_csv(not_empty, "not_empty")

2374

In [15]:
df.loc[
    (
        (df.loc[:, ~df.columns.isin(  ["url", "content",])] == "")
        | (df.loc[:, ~df.columns.isin(["url", "content",])].isnull())
    ).any(axis=1)
].shape[0]

4739

In [40]:
df[df['sub_title'] != ""].sum()

TypeError: can only concatenate str (not "int") to str

In [45]:
results = [{
    "total": df['url'].notnull().sum(),
    "page_title": df['page_title'].notnull().sum(),
    "subtitle": df['sub_title'].notnull().sum(),
    "introduction": df['introduction'].notnull().sum(),
    "summary_box": (df['summary_box'] != "").sum(),
    "content": (df['content'] != "").sum(),
    "accordion": (df['accordion'] != "").sum(),
}]
a = pd.DataFrame(results)
display(a)
# save_as_csv(a, "welche_spalten_haben_wie_viele_inhalte")

,total,page_title,subtitle,introduction,summary_box,content,accordion
0,4809,2503,425,1276,198,745,163


In [33]:
df['sub_title'].notnull().sum()

425

In [189]:
empty_except_page_title = df.loc[
    (
        (df.loc[:, ~df.columns.isin(["url", "page_title"])] == "")
        | (df.loc[:, ~df.columns.isin(["url", "page_title"])].isnull())
    ).all(axis=1)
    # & ~(
    #     (df["url"].str.contains("kunst"))
    #     | (df["url"].str.contains("vcard"))
    #     | (df["url"].str.contains("jobs"))
    #     | (df["url"].str.contains("report"))
    #     | (df["url"].str.contains("gallerySlide"))
    # )
]
empty_except_page_title.shape[0]
# save_as_csv(empty_except_page_title, "empty_except_page_title_with_gallerySlide")

3305

In [172]:
https_df_everything_empty = df.loc[
    (
        (df.loc[:, ~df.columns.isin(["url"])] == "") |
        (df.loc[:, ~df.columns.isin(["url"])].isnull())
    ).all(axis=1)
    & ~((df["url"].str.contains("kunst")) | (df["url"].str.contains("vcard"))) 
]
https_df_everything_empty.shape[0]
display(https_df_everything_empty)
# save_as_csv(https_df_everything_empty, "all_empty_not_containing_url_or_vcard")

,url,page_title,sub_title,introduction,summary_box,content,accordion
28,https://www.mobiliar.ch/versicherungen-und-vorsorge/generalagenturen,None,None,None,,,
32,https://www.mobiliar.ch/hub/wohnen,None,None,None,,,
43,https://jobs.mobiliar.ch/Jobs/All?DesignID=00&lang=ger,None,None,None,,,
57,https://report.mobiliar.ch/2022/de/,None,None,None,,,
80,https://my.mobiliar.ch/portal-rwc/?language=de,None,None,None,,,
...,...,...,...,...,...,...,...
4707,https://www.mobiliar.ch/hub/wohnen/inspiration,None,None,None,,,
4709,https://www.mobiliar.ch/hub/wohnen/ratgeberartikel,None,None,None,,,
4710,https://www.mobiliar.ch/hub/wohnen/videoartikel,None,None,None,,,
4732,https://www.mobiliar.ch/hub/wohnen/unser-haus/unser-haus-making-of,None,None,None,,,


In [121]:
def count_not_null_or_empty_of_X(pandas_df):
    col_list = pandas_df.columns.values.tolist()
    results = []
    for x in col_list:
        x_count = pandas_df[pandas_df[x].notna() & (pandas_df[x] != "")].shape[0]
        pct = round(x_count/pandas_df.shape[0]*100, 2)
        results.append({
            'column_name': x,
            'is_not_empty_count': x_count,
            'percentage': pct
        })
    return pd.DataFrame(results)

In [141]:
count_not_null_or_empty_of_X(df)
# save_as_csv(count_not_null_or_empty_of_X(df), "count_not_empty_by_column")
